In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
import glob

In [ ]:
IMAGESHAPE = [224, 224, 3]
training_data = 'chest_xray/train'
testing_data = 'chest_xray/test'


In [ ]:
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
for each_layer in vgg_model.layers:
	each_layer.trainable = False


In [ ]:
classes =glob.glob('/content/drive/MyDrive/Datasets/chest_xray/train/*')


In [ ]:
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)


In [ ]:
final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
final_model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
								shear_range = 0.2,
								zoom_range = 0.2,
								horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)


In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/chest_xray/train',
												target_size = (224, 224),
												batch_size = 4,
												class_mode = 'categorical')


Found 5216 images belonging to 2 classes.


In [ ]:
test_set = testing_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/chest_xray/test',
											target_size = (224, 224),
											batch_size = 4,
											class_mode = 'categorical')


Found 624 images belonging to 2 classes.


In [ ]:
fitted_model = final_model.fit_generator( 
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-13-c34fa144093e>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fitted_model = final_model.fit_generator(


Epoch 1/5
1304/1304 [==============================] - 3531s 3s/step - loss: 0.2709 - accuracy: 0.9227 - val_loss: 0.3392 - val_accuracy: 0.9183
Epoch 2/5
1304/1304 [==============================] - 3371s 3s/step - loss: 0.2419 - accuracy: 0.9454 - val_loss: 0.5788 - val_accuracy: 0.9038
Epoch 3/5
1304/1304 [==============================] - 3371s 3s/step - loss: 0.2183 - accuracy: 0.9548 - val_loss: 0.5382 - val_accuracy: 0.9135
Epoch 4/5
1304/1304 [==============================] - 3371s 3s/step - loss: 0.2487 - accuracy: 0.9548 - val_loss: 1.4611 - val_accuracy: 0.8750
Epoch 5/5
1304/1304 [==============================] - ETA: 0s - loss: 0.2109 - accuracy: 0.9626

In [ ]:
final_model.save('my_model.h5')


In [ ]:
from tensorflow.keras.preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np
model=load_model('my_model.h5') #Loading our model
img=image.load_img('/content/drive/MyDrive/Datasets/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg',target_size=(224,224))
imagee=image.img_to_array(img) #Converting the X-Ray into pixels
imagee=np.expand_dims(imagee, axis=0)
img_data=preprocess_input(imagee)
prediction=model.predict(img_data)
if prediction[0][0]>prediction[0][1]: #Printing the prediction of model.
	print('Person is safe.')
else:
	print('Person is affected with Pneumonia.')
print(f'Predictions: {prediction}')


1/1 [==============================] - 1s 744ms/step
Person is affected with Pneumonia.
Predictions: [[0. 1.]]


# New Section